# TEXT MINING for PRACTICE
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

In [2]:
# TEANAPS (https://github.com/fingeredman/teanaps)
#!git clone https://github.com/fingeredman/teanaps.git

In [3]:
#!ls

In [4]:
# TEANAPS 설치를 진행합니다.
# 설치 전 반드시 상단 메뉴에서 [런타임 > 런타임 초기화]를 클릭한 후 진행해주세요.
#!python "teanaps/teanaps_setup.py"

## WEEK 08-1. 텍스트 분류 (Text Classification)
- Python으로 텍스트 분류를 수행하는 방법에 대해 다룹니다.

---

### 1. 학습 데이터 준비하기

---

#### 1.1. TEANAPS 라이브러리 불러오기

---

In [5]:
from teanaps.nlp import MorphologicalAnalyzer
from teanaps.nlp import Processing
from teanaps.handler import FileHandler
from teanaps.text_analysis import TfidfCalculator

ma = MorphologicalAnalyzer()
tfidf = TfidfCalculator()
processing = Processing()
fh = FileHandler()

#### 1.2. 데이터 파일 불러오기

---

In [6]:
data_path = "data/chat_intent.txt"
data = fh.load_txt(data_path)

In [7]:
print(len(data))

611


In [8]:
data[:5]

[['몇시야', 'date'],
 ['몇시지', 'date'],
 ['몇시일까', 'date'],
 ['몇시게', 'date'],
 ['시간', 'date']]

#### 1.3. 데이터 전처리하기

---

In [9]:
intent_id_to_name = {}
intent_name_to_id = {}
intent_id = -1
temp_intent_name = None

data_list = []

index = 0
for query, intent_name in data:
    index += 1
    print(index, end="\r")
    
    # 클래스 이름이 다른경우 클래스 ID(intent_id)를 업데이트
    if temp_intent_name != intent_name:
        intent_id += 1
    temp_intent_name = intent_name
    intent_id_to_name[intent_id] = intent_name
    intent_name_to_id[intent_name] = intent_id
    
    # 형태소분석 후 data_list 추가
    query_lower = query.lower()
    pos_result = ma.parse(query_lower)
    data_list.append(processing.get_plain_text(pos_result, tag=False))

In [10]:
intent_id_to_name

{0: 'date', 1: 'weather', 2: 'restraunt'}

In [11]:
intent_name_to_id

{'date': 0, 'weather': 1, 'restraunt': 2}

In [12]:
data_list[:10]

['몇 시야',
 '몇 시지',
 '몇 시일 까',
 '몇 시 게',
 '시간',
 '몇 시',
 '시간 알려줘',
 '지금 몇 시야',
 '지금 몇 시지',
 '몇 시인 지 알 아']

#### 1.4. 텍스트 데이터 임베딩: TF-IDF Matrix

---

In [13]:
from sklearn.model_selection import train_test_split

# TF-IDF 계산
tfidf.calculation_tfidf(data_list, 
                        tf_vectorizer_path="data/tf_vectorizer", 
                        tfidf_vectorizer_path="data/tfidf_vectorizer")
tfidf_matrix = tfidf.get_tfidf_matrix().values[:]

# 정답 label 불러오기
label_list = [intent_name_to_id[intent_name] for _, intent_name in data]

# 학습데이터/평가데이터 분리
x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, label_list, test_size=0.20, random_state=None)

### 2. 학습하기

---

#### 2.1. 학습: 랜덤 포레스트 (Random Forest)

---

In [14]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)         # 학습
score = random_forest.score(x_test, y_test) # 평가
print("Accuracy:", score)

# 학습모델 저장
fh.save_data("data/rf_model", random_forest)

Accuracy: 0.983739837398374


#### 2.2. 학습: 서포트 벡터 머신 (SVM)

---

In [15]:
from sklearn.svm import SVC

svm_model = SVC(probability=True)
svm_model.fit(x_train, y_train)         # 학습
score = svm_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/svm_model", svm_model)

Accuracy : 0.991869918699187


### 3. 학습모델 활용하기

---

In [18]:
query = "오늘 며칠이지?"
#query = "근처 맛집 추천해줘"
#query = "지금 비와?"

query_lower = query.lower()
pos_result = ma.parse(query_lower)
input_vector = tfidf.get_tfidf_vector(processing.get_plain_text(pos_result, tag=False), 
                                      tfidf_vectorizer_path="data/tfidf_vectorizer")
# 학습모델 불러오기
model = fh.load_data("data/rf_model")
#model = fh.load_data("data/svm_model")
# 분류수행
intent_prob_list = model.predict_proba([input_vector]).tolist()[0]

intent_list = [(intent_id_to_name[i], i, r) for i, r in enumerate(intent_prob_list)]
intent_list.sort(key=lambda elem: elem[2], reverse=True)
for intent_no, intent in enumerate(intent_list):
    print(intent_no, intent)

0 ('date', 0, 0.997967387846658)
1 ('weather', 1, 0.0014942971107549188)
2 ('restraunt', 2, 0.0005383150425869258)


## WEEK 07-2. 감성분석
- Python의 TEANAPS 라이브러리를 활용해 감성분석을 수행하는 방법에 대해 다룹니다.

---

### 1. 감성분석 수행방법 알아보기: TEANAPS

---

#### 1.1. TEANAPS 패키지로 감성분석 수행하기

---

In [19]:
from teanaps.text_analysis import SentimentAnalysis

#senti = SentimentAnalysis(model_path="/model", kobert_path="/kobert")
senti = SentimentAnalysis()

In [20]:
sentence = "늘 배우고 배푸는 자세가 필요합니다."
result = senti.tag(sentence, neutral_th=0.3)
print(result)

((0.0595, 0.9543), 'positive')


In [21]:
sentence = "과한 욕심은 주변 사람들에게 피해를 줍니다."
result = senti.tag(sentence, neutral_th=0.3)
print(result)

((0.8715, 0.1076), 'negative')


#### 1.2. 형태소별 가중치 출력하기

---

In [22]:
sentence = "늘 배우고 배푸는 자세가 필요합니다."
token_list, weight_list = senti.get_weight(sentence)
print(token_list)
print(weight_list)

[' 늘', ' 배우', '고', ' 배', '푸', '는', ' 자세', '가', ' 필요', '합니다', ' ', '.']
[0.072522074, 0.08697342, 0.052703843, 0.051040735, 0.0606895, 0.05134341, 0.05213573, 0.08644837, 0.078125894, 0.079360135, 0, 0.079488374]


In [23]:
sentence = "과한 욕심은 주변 사람들에게 피해를 줍니다."
token_list, weight_list = senti.get_weight(sentence)
print(token_list)
print(weight_list)

[' ', '과', '한', ' 욕심', '은', ' 주변', ' 사람들', '에게', ' 피해를', ' ', '줍', '니다', ' ', '.']
[0, 0.020344315, 0.024879746, 0.02612342, 0.03615231, 0.048542265, 0.06707654, 0.0936653, 0.07649707, 0, 0.08189902, 0.08962273, 0, 0.07841993]


#### 1.3. 형태소별 가중치 시각화하기: 강조표현

---

In [26]:
sentence = "늘 배우고 배푸는 자세가 필요합니다."
senti.draw_sentence_weight(sentence)

In [27]:
sentence = "과한 욕심은 주변 사람들에게 피해를 줍니다."
senti.draw_sentence_weight(sentence)

#### 1.5. 어절별 감성분석 수행하기

---

In [28]:
sentence = "욕심쟁이에게 스트레스 받으며 살다가 떠나고나니 너무 행복해요!"
phrase_token_weight_list, token_list, weight_list = senti.get_sentiment_parse(sentence, neutral_th=0.5)
#phrase_token_weight_list, token_list, weight_list = senti.get_sentiment_parse(sentence, neutral_th=0.5, model_path="/ner_model")
print(phrase_token_weight_list)
print(token_list)
print(weight_list)

[(((0.5991, 0.3836), 'neutral'), '욕심쟁이에게', [[('욕심쟁이', 'NNG', 'UN', (0, 4))], [('에게', 'JKB', 'UN', (4, 6))]]), (((0.9147, 0.0828), 'negative'), '스트레스 받으며', [[('스트레스', 'NNG', 'UN', (7, 11)), ('받', 'VV', 'UN', (12, 13))], [('으며', 'EC', 'UN', (13, 15))]]), (((0.9047, 0.0953), 'negative'), '살다가', [[('살', 'VV', 'UN', (16, 17))], [('다가', 'EC', 'UN', (17, 19))]]), (((0.8306, 0.1751), 'negative'), '떠나고', [[('떠나', 'VV', 'UN', (20, 22))], [('고', 'EC', 'UN', (22, 23))]]), (((0.453, 0.5296), 'neutral'), '나니', [[('나', 'VX', 'UN', (23, 24))], [('니', 'EC', 'UN', (24, 25))]]), (((0.1065, 0.8982), 'positive'), '너무 행복해요!', [[('너무', 'MAG', 'UN', (26, 28))], [('행복', 'NNG', 'UN', (29, 31))], [('해요', 'XSV+EF', 'UN', (31, 33)), ('!', 'SW', 'UN', (33, 34))]])]
[' 욕심', '쟁', '이', '에게', ' 스트레스', ' 받으며', ' 살', '다', '가', ' 떠나', '고', ' 나', '니', ' 너무', ' 행복', '해', '요', ' ', '!']
[0, 0, 0, 0, -0.2424436, -0.20117857, -0.16506892, -0.16892226, -0.27025366, -0.16876356, -0.33119142, 0, 0, 0.15942541, 0.13346915, 0.11855

In [29]:
#sentence = "욕심쟁이에게 스트레스 받으며 살다가 떠나고나니 너무 행복해요!"
#token_list = [' 욕심', '쟁', '이', '에게', ' 스트레스', ' 받으며', ' 살', '다', '가', ' 떠나', '고', ' 나', '니', ' 너무', ' 행복', '해', '요', ' ', '!']
#weight_list = [0, 0, 0, 0, -0.2424436, -0.20117857, -0.16506892, -0.16892226, -0.27025366, -0.16876356, -0.33119142, 0, 0, 0.15942541, 0.13346915, 0.11855107, 0.15605149, 0, 0.11754697]
senti.draw_sentiment_parse(token_list, weight_list)

### Assignment 05. 분류기 성능 향상시키기
> - 본 실습자료를 활용하여 분류기 성능을 향상시킬 수 있는 방법을 찾아 적용하고 개선된 분류기 성능평가 결과(accuracy)를 제출합니다.
> - 과제를 수행하는 방향성은 두 가지가 있습니다. (2번 우선)
> > 1. 알고리즘 개선 : 분류 알고리즘의 옵션을 변경하거나 실습 외 분류 알고리즘 활용 등
> > 2. 학습데이터 정제 : 학습데이터 중 유사한 클래스를 통합하거나 학습데이터를 추가/수정하는 방법 등
> - 학습데이터는 "data/chat_intent_total.txt" 파일을 활용합니다.
> - 분류 알고리즘을 어느 것을 사용해도 무방합니다. 다양한 알고리즘에 대해 테스트해보시고 가장 성능이 좋은것으로 제출하시면 됩니다.
> - 실습자료 외 분류 알고리즘을 활용해도 무방합니다.
> - 과제 제출 시 Jupyter Notebook 파일에 어떤 방법으로 성능을 향상시켰는지 간단히 주석을 작성해주세요.
> - 테스트/학습 데이터의 비율은 8:2를 기준으로 합니다. - train_test_split(..., test_size=0.20, ...)

---